<a href="https://colab.research.google.com/github/deacs11/CrewAI_Pantry_to_Plate_Recipe_-_Meal_Planner_Crew/blob/main/CrewAI_Pantry_to_Plate_Recipe_%26_Meal_Planner_Crew.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title 1. Install necessary libraries
# Installs crewai, tools, OpenAI client, Colab module, and search tool
!pip install crewai crewai-tools langchain-openai google-colab duckduckgo-search==5.3.1b1 -q

print("Library installation completed!")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.3/265.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.4/561.4 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 9.5 MB/s eta 

In [2]:
# @title 2. Import modules and configure API keys from secrets
import os
from google.colab import userdata # To read Colab secrets
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool, WebsiteSearchTool # Tools for recipe searching
from langchain_openai import ChatOpenAI
import json # May be useful for structured data

# --- API key configuration from Colab secrets ---
print("Attempting to load API keys from Secrets...")
try:
    # Reads the OpenAI key from the secret 'OPENAI_API_KEY'
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
    # Reads the Serper key from the secret 'SERPER_API_KEY'
    os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')
    print("-> API key environment variables set successfully.")
except Exception as e:
    print(f"!!! Error loading API keys from secrets: {e}")
    print("!!! Make sure you have set them correctly in the Colab Secrets panel")
    print("!!! and enabled 'Notebook access' for both.")

# Debug block to verify keys
print("-" * 20)
retrieved_key = os.environ.get("OPENAI_API_KEY")
if retrieved_key:
    print(f"OpenAI key FOUND in environment: '{retrieved_key[:5]}...{retrieved_key[-4:]}'")
else:
    print("!!! OpenAI key NOT FOUND.")
print("-" * 20)
retrieved_serper_key = os.environ.get("SERPER_API_KEY")
if retrieved_serper_key:
     print(f"Serper key FOUND in environment: '{retrieved_serper_key[:5]}...{retrieved_serper_key[-4:]}'")
else:
    print("--- Serper key NOT FOUND (required for RecipeDetailFetcher agent).")
print("-" * 20)

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_config.py:323: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_fields.py:198: UserWarning: Field name "schema" in "DatabricksQueryToolSchema" shadows an attribute in parent "BaseModel"
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:617: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
/usr/local/lib/python3.11/dist-packages/crewai_t

Attempting to load API keys from Secrets...
-> API key environment variables set successfully.
--------------------
OpenAI key FOUND in environment: 'sk-pr...DKkA'
--------------------
Serper key FOUND in environment: '7354c...dc56'
--------------------


In [5]:
# @title 3. Define pantry inventory and preferences

# --- MODIFY THESE VARIABLES WITH AVAILABLE INGREDIENTS AND PREFERENCES ---
# List ingredients clearly, one per line or comma-separated is fine.
available_ingredients = """
Chicken Breasts (2)
Broccoli (1 head)
Onion (1)
Garlic (3 cloves)
Soy Sauce
Sesame Oil
Rice Vinegar
Brown Sugar
Pasta (Spaghetti)
Canned Tomatoes (1 can)
Olive Oil
Salt, Pepper
Dried Oregano
Eggs (4)
Milk
Flour (All-purpose)
Butter
Carrots (2)
"""

# Specify dietary restrictions or strong preferences, comma-separated or list. Use "None" if none.
dietary_restrictions = "Lactose intollerant" # Examples: "Vegetarian", "Gluten-Free", "No Nuts"

# Specify desired meal type(s)
desired_meal_types = "Dinner, Quick Lunch" # Examples: "Dinner", "Breakfast", "Any"
# -----------------------------------------------------------------------------

print("User Inputs Defined:")
print("--- Available Ingredients ---")
print(available_ingredients.strip())
print("-----------------------------")
print(f"Dietary Restrictions: {dietary_restrictions}")
print(f"Desired Meal Types: {desired_meal_types}")

User Inputs Defined:
--- Available Ingredients ---
Chicken Breasts (2)
Broccoli (1 head)
Onion (1)
Garlic (3 cloves)
Soy Sauce
Sesame Oil
Rice Vinegar
Brown Sugar
Pasta (Spaghetti)
Canned Tomatoes (1 can)
Olive Oil
Salt, Pepper
Dried Oregano
Eggs (4)
Milk
Flour (All-purpose)
Butter
Carrots (2)
-----------------------------
Dietary Restrictions: Lactose intollerant
Desired Meal Types: Dinner, Quick Lunch


In [6]:
# @title 4. Select LLM and initialize tools

# --- Choose the language model (LLM) ---
try:
    # GPT-4 recommended for better recipe understanding and generation
    llm = ChatOpenAI(model="gpt-4-turbo")
    # llm = ChatOpenAI(model="gpt-3.5-turbo") # Might be less creative or accurate
    print(f"LLM ({llm.model_name}) initialized successfully.")
except Exception as e:
    print(f"!!! Error initializing ChatOpenAI: {e}")
    print("!!! Verify OpenAI API key.")
    raise

# --- Initialize tools ---
# Tools needed for the RecipeDetailFetcher agent
search_tool = None
web_reader_tool = None
try:
    search_tool = SerperDevTool()
    print("SerperDevTool initialized.")
    # WebsiteSearchTool might be helpful if the search returns direct recipe links
    web_reader_tool = WebsiteSearchTool()
    print("WebsiteSearchTool initialized.")
except Exception as e:
    print(f"--- Warning: Error initializing tools: {e}. Recipe detail fetching might be limited.")

available_tools = [tool for tool in [search_tool, web_reader_tool] if tool is not None]
print(f"Tools actually available for research: {[t.name for t in available_tools] if available_tools else 'None'}")

LLM (gpt-4-turbo) initialized successfully.
SerperDevTool initialized.
WebsiteSearchTool initialized.
Tools actually available for research: ['Search the internet with Serper', 'Search in a specific website']


In [11]:
# @title 5. Define agents for pantry-to-plate crew (Strict Inventory Version)

print("Defining pantry-to-plate agents (Strict Inventory Version)...")

if 'llm' not in locals() or llm is None:
     raise ValueError("LLM not initialized.")

# --- Agent 1: Ingredient Analyzer (No changes needed) ---
ingredient_analyzer = Agent(
    role='Pantry Inventory Specialist',
    goal='Parse and structure the list of available ingredients. Normalize names slightly if possible and identify the core components (proteins, vegetables, staples, condiments).',
    backstory=(
        "You have an encyclopedic knowledge of food ingredients and kitchen staples. You can quickly scan a list, understand what's available, "
        "and categorize items logically to facilitate recipe brainstorming."
    ),
    tools=[], # Relies on LLM understanding
    llm=llm,
    allow_delegation=False,
    verbose=True,
    max_iter=3
)
print("- Agent 'ingredient_analyzer' defined.")

# --- Agent 2: Recipe Ideator (No major changes needed) ---
recipe_ideator = Agent(
    role='Creative Chef / Recipe Brainstormer',
    goal=(f"Based on the analyzed list of available core ingredients and considering the dietary restrictions ('{dietary_restrictions}') "
          f"and desired meal types ('{desired_meal_types}'), brainstorm 2-3 distinct and suitable recipe *ideas* (names or concepts) "
          f"that prominently feature the available ingredients."),
    backstory=(
        "You are a creative and practical chef who excels at improvising with available ingredients. You understand flavor pairings and dietary needs. "
        "Your goal is to suggest appealing and feasible meal ideas based *only* on what's on hand, ensuring they meet the user's constraints." # Added emphasis
    ),
    tools=[], # Primarily creative generation based on context
    llm=llm,
    allow_delegation=False,
    verbose=True,
    max_iter=5
)
print("- Agent 'recipe_ideator' defined.")

# --- Agent 3: Recipe Detail Generator (MODIFIED) ---
# Renamed for clarity, goal significantly changed
recipe_detail_generator = Agent(
    role='Constrained Recipe Crafter',
    goal=("For each recipe *idea* provided, generate plausible cooking instructions using **ONLY** the ingredients listed as 'available' in the initial inventory context. "
          "Assume basic staples like salt, pepper, and a standard cooking oil (like olive or vegetable) are available even if not explicitly listed, but add **NO OTHER** external ingredients. "
          "Estimate reasonable quantities based on the idea (e.g., use 'the chicken breasts', 'half the onion'). "
          "Provide a list of *only the available ingredients used* for that specific recipe and basic step-by-step instructions."),
    backstory=(
        "You are a master of resourcefulness in the kitchen. You can devise clear cooking instructions using *only* a strictly limited set of ingredients. "
        "You focus on creating simple, achievable steps that utilize the provided items effectively, without requiring anything extra (beyond absolute basics like salt/pepper/oil)."
    ),
    tools=[], # No external search needed; relies on generation & context
    llm=llm,
    allow_delegation=False,
    verbose=True,
    max_iter=7 # Might need iteration to generate plausible steps within constraints
)
# Changed variable name for clarity
print("- Agent 'recipe_detail_generator' (Modified) defined.")

# --- Agent 4: Recipe Constraint Verifier (MODIFIED) ---
# Renamed and role changed from ShoppingListCompiler
recipe_constraint_verifier = Agent(
    role='Recipe Ingredient Auditor',
    goal=("Verify that the ingredients listed for each generated recipe instruction set (provided in context) strictly adhere to the initial list of 'available ingredients' (also provided in context). "
          "Allow for the assumed staples (salt, pepper, basic cooking oil). Flag any discrepancies or ingredients used in the instructions that were *not* on the original available list. "
          "Confirm compliance if no discrepancies are found."),
    backstory=(
        "You are a meticulous auditor with a keen eye for detail. You cross-reference ingredient lists precisely. Your job is to ensure that the generated recipes "
        "faithfully adhere to the constraint of using only the initially available items (plus staples). You report deviations clearly."
    ),
    tools=[], # Performs comparison based on context
    llm=llm,
    allow_delegation=False,
    verbose=True,
    max_iter=3
)
# Changed variable name for clarity
print("- Agent 'recipe_constraint_verifier' (Modified) defined.")

# --- Agent 5: Meal Plan Presenter (Slight modification needed later in task) ---
meal_plan_presenter = Agent(
    role='Meal Planner & Recipe Organizer',
    goal='Format the final output clearly. Present the suggested recipe(s) including their generated ingredient lists (using available items) and instructions. Include the verification result regarding ingredient constraints. Ensure the output is easy to read and use.',
    backstory=(
        "You specialize in creating clear and actionable meal plans and recipe cards. You take the generated recipe details and verification status and organize them "
        "logically so the user knows what they can cook with their current inventory. Presentation and usability are key."
    ),
    tools=[],
    llm=llm,
    allow_delegation=False,
    verbose=True,
    max_iter=3
)
print("- Agent 'meal_plan_presenter' defined.")

print("All pantry-to-plate agents (Strict Inventory Version) defined.")

Defining pantry-to-plate agents (Strict Inventory Version)...
- Agent 'ingredient_analyzer' defined.
- Agent 'recipe_ideator' defined.
- Agent 'recipe_detail_generator' (Modified) defined.
- Agent 'recipe_constraint_verifier' (Modified) defined.
- Agent 'meal_plan_presenter' defined.
All pantry-to-plate agents (Strict Inventory Version) defined.


In [12]:
# @title 6. Define tasks for pantry-to-plate crew (Strict Inventory Version)

print("Defining pantry-to-plate tasks (Strict Inventory Version)...")

# Verify agents are defined using the NEW variable names
if 'ingredient_analyzer' not in locals() or \
   'recipe_ideator' not in locals() or \
   'recipe_detail_generator' not in locals() or \
   'recipe_constraint_verifier' not in locals() or \
   'meal_plan_presenter' not in locals():
    raise ValueError("One or more required agents (using updated names) are not defined.")

# --- Task 1: Analyze Available Ingredients (No changes needed) ---
task_analyze_ingredients = Task(
    description=(
        f"Parse the following list of available ingredients provided by the user:\n---\n{available_ingredients}\n---\n"
        f"Structure this list, perhaps identifying core items. Output the structured list of available ingredients for context."
    ),
    agent=ingredient_analyzer,
    expected_output=(
        "A structured list or summary of the available ingredients, potentially categorized or cleaned for clarity. This will serve as the 'pantry inventory' context for subsequent tasks."
    )
)
print("- Task 'task_analyze_ingredients' defined.")

# --- Task 2: Brainstorm Recipe Ideas (No changes needed) ---
task_ideate_recipes = Task(
    description=(
        f"Based on the structured list of available ingredients (from context) and considering the user's constraints: "
        f"Dietary Restrictions: '{dietary_restrictions}', Desired Meal Types: '{desired_meal_types}'. "
        f"Brainstorm and list 2-3 distinct recipe *ideas* (e.g., 'Chicken & Broccoli Dish', 'Simple Pasta with Tomato', 'Basic Omelette') "
        f"that make good use of the core available ingredients and meet the constraints."
    ),
    agent=recipe_ideator,
    expected_output=(
        "A list of 2-3 distinct recipe ideas (names or brief concepts) suitable for the ingredients and constraints."
    ),
    context=[task_analyze_ingredients] # Needs the analyzed ingredients
)
print("- Task 'task_ideate_recipes' defined.")

# --- Task 3: Generate Recipe Details (MODIFIED) ---
# Uses the new agent and has modified instructions
task_generate_recipe_details = Task(
    description=(
        "For each recipe idea suggested (context provided), generate plausible cooking instructions. **Crucially, use ONLY ingredients from the initial 'available ingredients' list** (also provided in context). "
        "You may assume salt, pepper, and basic cooking oil are available. Do not include any other external ingredients. "
        "Estimate reasonable quantities based on the available items (e.g., 'use the 2 chicken breasts'). "
        "Output should include for each recipe idea:\n"
        "1. Recipe Name/Idea.\n"
        "2. A list of **only the available ingredients used** for this recipe.\n"
        "3. Basic, step-by-step cooking instructions using only those ingredients."
    ),
    # Assign task to the renamed/modified agent
    agent=recipe_detail_generator,
    expected_output=(
        "Structured details for 2-3 recipes. Each recipe entry must include:\n"
        "- Recipe Name/Idea\n"
        "- List of **Available Ingredients Used** (strictly from initial list + staples)\n"
        "- Basic Cooking Instructions (step-by-step, using only listed ingredients)"
    ),
    # Needs recipe ideas and the initial ingredient list
    context=[task_analyze_ingredients, task_ideate_recipes]
)
# Changed variable name for clarity
print("- Task 'task_generate_recipe_details' (Modified) defined.")

# --- Task 4: Verify Recipe Constraints (MODIFIED) ---
# Replaces the shopping list task
task_verify_constraints = Task(
    description=(
        "Carefully review the 'Available Ingredients Used' list generated for each recipe in the previous task (context provided). "
        "Compare this list against the *initial* list of available ingredients (also in context). "
        "Verify if the recipes strictly used only the initially available items (allowing for salt, pepper, basic cooking oil). "
        "Report the result of this verification for each recipe. State clearly if compliance is confirmed or list any ingredients used that were NOT on the original list."
    ),
    # Assign task to the renamed/modified agent
    agent=recipe_constraint_verifier,
    expected_output=(
        "A verification summary. For each recipe checked, state either 'Compliance Confirmed: Uses only available ingredients.' or 'Discrepancy Found: Uses [list violating ingredient(s)] which were not on the original list.'."
    ),
    # Needs the generated recipe details and the initial ingredient list
    context=[task_analyze_ingredients, task_generate_recipe_details]
)
# Changed variable name for clarity
print("- Task 'task_verify_constraints' (Modified) defined.")

# --- Task 5: Present Recipes and Verification (MODIFIED) ---
task_present_plan = Task(
    description=(
        "Compile the final output. Create a well-formatted document including:\n"
        "1. The suggested Recipe(s) with their generated 'Available Ingredients Used' list and step-by-step instructions (from Task 3 context).\n"
        "2. The result of the constraint verification (from Task 4 context) for each recipe.\n"
        "Use Markdown for clear headings and lists. Add a brief intro and a reminder about checking allergy info."
    ),
    agent=meal_plan_presenter,
    expected_output=(
        "A final, formatted document (Markdown recommended) containing:\n"
        "- A brief introduction.\n"
        "- Section(s) for each suggested recipe detailing **only available ingredients used** and instructions.\n"
        "- A section or note confirming the result of the ingredient constraint verification.\n"
        "- A concluding reminder about checking details/allergies."
    ),
    # Needs the generated recipe details and the verification results
    context=[task_generate_recipe_details, task_verify_constraints]
)
print("- Task 'task_present_plan' (Modified Context) defined.")

print("All pantry-to-plate tasks (Strict Inventory Version) defined.")

Defining pantry-to-plate tasks (Strict Inventory Version)...
- Task 'task_analyze_ingredients' defined.
- Task 'task_ideate_recipes' defined.
- Task 'task_generate_recipe_details' (Modified) defined.
- Task 'task_verify_constraints' (Modified) defined.
- Task 'task_present_plan' (Modified Context) defined.
All pantry-to-plate tasks (Strict Inventory Version) defined.


In [13]:
# @title 7. Create and run pantry-to-plate crew

print("Creating the pantry-to-plate crew...")

# Verify all components are ready
if 'ingredient_analyzer' not in locals() or \
   'recipe_ideator' not in locals() or \
   'recipe_detail_fetcher' not in locals() or \
   'shopping_list_compiler' not in locals() or \
   'meal_plan_presenter' not in locals() or \
   'task_analyze_ingredients' not in locals() or \
   'task_ideate_recipes' not in locals() or \
   'task_get_recipe_details' not in locals() or \
   'task_compile_shopping_list' not in locals() or \
   'task_present_plan' not in locals():
    raise ValueError("Missing agents or tasks needed to create the Pantry-to-Plate Crew.")

# Assemble the Crew
pantry_to_plate_crew = Crew(
    agents=[ingredient_analyzer, recipe_ideator, recipe_detail_fetcher, shopping_list_compiler, meal_plan_presenter],
    tasks=[task_analyze_ingredients, task_ideate_recipes, task_get_recipe_details, task_compile_shopping_list, task_present_plan],
    process=Process.sequential,
    memory=True,
    cache=True,
    verbose=True
)

print("Pantry-to-Plate Crew created. Starting the process with kickoff()...")

# Define initial inputs (mainly for context, as ingredients are in task desc)
crew_inputs = {
    'initial_ingredients': available_ingredients,
    'dietary_needs': dietary_restrictions,
    'meal_preference': desired_meal_types
}

# Kick off the crew's work
result = pantry_to_plate_crew.kickoff(inputs=crew_inputs)

print("\n\n*******************************************")
print("   PANTRY-TO-PLATE CREW FINISHED WORK!   ")
print("*******************************************")

Creating the pantry-to-plate crew...
Pantry-to-Plate Crew created. Starting the process with kickoff()...


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 80be4625-686d-4900-be16-4abc591eb83f                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 468907ef-ab1d-4da8-980e-399fd3013064
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 468907ef-ab1d-4da8-980e-399fd3013064
       Status: Executing Task...
    └── 🤖 Agent: Pantry Inventory Specialist
            Status: In Progress

# Agent: Pantry Inventory Specialist
## Task: Parse the following list of available ingredients provided by the user:
---

Chicken Breasts (2)
Broccoli (1 head)
Onion (1)
Garlic (3 cloves)
Soy Sauce
Sesame Oil
Rice Vinegar
Brown Sugar
Pasta (Spaghetti)
Canned Tomatoes (1 can)
Olive Oil
Salt, Pepper
Dried Oregano
Eggs (4)
Milk
Flour (All-purpose)
Butter
Carrots (2)

---
Structure this list, perhaps identifying core items. Output the structured list of available ingredients for context.


...

🚀 Crew: crew
├── 📋 Task: 468907ef-ab1d-4da8-980e-399fd3013064
│      Assigned to: Pantry Inventory Specialist
│      Status: ✅ Completed
│   └── 🤖 Agent: Pantry Inventory Specialist
│           Status: ✅ Completed
├── 📋 Task: 2d0bc0e9-4074-4784-8dbe-8f2521e7af63
│      Assigned to: Creative Chef / Recipe Brainstormer
│      Status: ✅ Completed
│   └── 🤖 Agent: Creative Chef / Recipe Brainstormer
│           Status: ✅ Completed
├── 📋 Task: c537bf98-5c62-4de5-95a2-554fb69eabef
│      Assigned to: Recipe Researcher and Detailer
│      Status: ✅ Completed
│   └── 🤖 Agent: Recipe Researcher and Detailer
│           Status: ✅ Completed
│       └── 🔧 Used Search the internet with Serper (4)
├── 📋 Task: 365a74fb-bbc0-498a-a30e-a5c874568ed5
│      Assigned to: Kitchen Inventory Manager
│      Status: ✅ Completed
│   └── 🤖 Agent: Kitchen Inventory Manager
│           Status: ✅ Completed
└── 📋 Task: 313f592b-d0a8-496a-8348-636ae2263e43
       Status: Executing Task...
    └── 🤖 Agent: Meal Planner & Recipe Organizer
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 468907ef-ab1d-4da8-980e-399fd3013064
│      Assigned to: Pantry Inventory Specialist
│      Status: ✅ Completed
│   └── 🤖 Agent: Pantry Inventory Specialist
│           Status: ✅ Completed
├── 📋 Task: 2d0bc0e9-4074-4784-8dbe-8f2521e7af63
│      Assigned to: Creative Chef / Recipe Brainstormer
│      Status: ✅ Completed
│   └── 🤖 Agent: Creative Chef / Recipe Brainstormer
│           Status: ✅ Completed
├── 📋 Task: c537bf98-5c62-4de5-95a2-554fb69eabef
│      Assigned to: Recipe Researcher and Detailer
│      Status: ✅ Completed
│   └── 🤖 Agent: Recipe Researcher and Detailer
│           Status: ✅ Completed
│       └── 🔧 Used Search the internet with Serper (4)
├── 📋 Task: 365a74fb-bbc0-498a-a30e-a5c874568ed5
│      Assigned to: Kitchen Inventory Manager
│      Status: ✅ Completed
│   └── 🤖 Agent: Kitchen Inventory Manager
│           Status: ✅ Completed
└── 📋 Task: 313f592b-d0a8-496a-8348-636ae2263e43
       Assigned to: Meal Planner & Recipe Organizer
       Status: ✅ Completed
    └── 🤖 Agent: Meal Planner & Recipe Organizer
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 313f592b-d0a8-496a-8348-636ae2263e43                                                                     │
│  Agent: Meal Planner & Recipe Organizer                                                                         │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 80be4625-686d-4900-be16-4abc591eb83f                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯



*******************************************
   PANTRY-TO-PLATE CREW FINISHED WORK!   
*******************************************


In [14]:
# @title 8. Display final recipes and shopping list

print("\n\n###################################################")
print("## AI-Generated Recipe Ideas & Shopping List:")
print("## Reminder: Double-check quantities & allergy info!")
print("###################################################\n")

if 'result' in locals():
    # Print the final output from the 'meal_plan_presenter' agent
    print(result)
else:
    print("!!! The 'result' variable is not defined. Ensure Cell 7 executed correctly.")

# Add final reminder
print("\n\n"+"-"*60)
print("Reminder: Please double-check recipe quantities and ensure ingredients meet")
print("your specific dietary needs and potential allergies.")
print("-"*60)



###################################################
## AI-Generated Recipe Ideas & Shopping List:
## Reminder: Double-check quantities & allergy info!
###################################################

# Meal Plan and Recipes

Welcome to your customized meal planning guide. Below you will find detailed recipes tailored to utilize only the ingredients available in your kitchen. Please review the recipes carefully and check each ingredient for potential allergens before preparation.

## Recipe Details

### 1. Chicken and Broccoli Stir-Fry
- **Available Ingredients Used:**
  - Chicken breasts, thinly sliced
  - Broccoli, cut into florets
  - Onion, sliced
  - Garlic, minced

- **Instructions:**
  1. Heat sesame oil in a large skillet over medium-high heat.
  2. Add the chicken slices and stir-fry until they start to brown.
  3. Add the onion and garlic, cooking until soft.
  4. Add the broccoli florets and continue to stir-fry for about 5 minutes.
  5. Stir in soy sauce and brown suga